In [1]:
from ctypes import *
import time

In [15]:
dledi = CDLL("SDK/NikonDlediDriver.dll")

In [16]:
uiDeviceCount = create_string_buffer(1)
ppiDeviceTypeList = create_string_buffer(1)

In [17]:
dledi.DLEDI_GetDeviceList(uiDeviceCount, ppiDeviceTypeList)
print("Found", ord(uiDeviceCount.value), "device(s)")

Found 1 device(s)


In [18]:
uiDeviceHandle = create_string_buffer(1)
uiErrMsgMaxSize = create_string_buffer(1)
pwszErrMsg = create_string_buffer(255)

In [19]:
# connect to DLEDI, return 0 if success
ret = dledi.DLEDI_Open(c_uint32(0), uiDeviceHandle, uiErrMsgMaxSize, pwszErrMsg)
if ret==0 :
    print("connected to device")
else:
    print("fail to connect")

connected to device


In [20]:
# uiDataUsageMask
DLEDI_DATA_MASK_OFF = 0x0000000000000000
DLEDI_DATA_MASK_FULL= 0xffffffffffffffff
DLEDI_DATA_MASK_SWITCH_1 = 0x0000000000000001
DLEDI_DATA_MASK_SWITCH_2 = 0x0000000000000002
DLEDI_DATA_MASK_SWITCH_3 = 0x0000000000000004
DLEDI_DATA_MASK_SWITCH_4 = 0x0000000000000008
DLEDI_DATA_MASK_SWITCH = (DLEDI_DATA_MASK_SWITCH_1 | DLEDI_DATA_MASK_SWITCH_2 | DLEDI_DATA_MASK_SWITCH_3 | DLEDI_DATA_MASK_SWITCH_4)
DLEDI_DATA_MASK_INTENSITY_1 = 0x0000000000000010
DLEDI_DATA_MASK_INTENSITY_2 = 0x0000000000000020
DLEDI_DATA_MASK_INTENSITY_3 = 0x0000000000000040
DLEDI_DATA_MASK_INTENSITY_4 = 0x0000000000000080
DLEDI_DATA_MASK_INTENSITY = (DLEDI_DATA_MASK_INTENSITY_1 | DLEDI_DATA_MASK_INTENSITY_2 | DLEDI_DATA_MASK_INTENSITY_3 | DLEDI_DATA_MASK_INTENSITY_4)

In [414]:
DLEDI_LED_MAX = 4 
class Switch(Structure):
    _fields_ = [("Switch", c_bool)]
    
class Intensity(Structure):
    _fields_ = [("Intensity", c_int32)]
    
class DLEDI_Data(Structure):
    _fields_ = [("uiDataUsageMask", c_uint64),
              ("bSwitch", Switch * DLEDI_LED_MAX),
              ("iIntensity", Intensity * DLEDI_LED_MAX)]

In [447]:
sData = DLEDI_Data()
sData.uiDataUsageMask = DLEDI_DATA_MASK_SWITCH | DLEDI_DATA_MASK_INTENSITY
dledi.DLEDI_DataGet(ord(uiDeviceHandle.value), byref(sData))
print("The LED intensity for each channel: ", end="")
for i in range(0, DLEDI_LED_MAX):
    print(sData.iIntensity[i].Intensity, end=",")
print()
print("The LED status for each channel: ", end="")
for i in range(0, DLEDI_LED_MAX):
    print(sData.bSwitch[i].Switch, end=",")

The LED intensity for each channel: 7,1,10,5,
The LED status for each channel: False,False,False,True,

In [456]:
# Set LED Intensity
sData = DLEDI_Data()
sDataOut = DLEDI_Data()
sData.bSwitch[0].Switch = c_bool(False)
sData.bSwitch[1].Switch = c_bool(True)
sData.bSwitch[2].Switch = c_bool(False)
sData.bSwitch[3].Switch = c_bool(False)
sData.iIntensity[0].Intensity = 0
sData.iIntensity[1].Intensity = 1
sData.iIntensity[2].Intensity = 2
sData.iIntensity[3].Intensity = 3
sData.uiDataUsageMask = DLEDI_DATA_MASK_SWITCH | DLEDI_DATA_MASK_INTENSITY
dledi.DLEDI_DataSet(ord(uiDeviceHandle.value), byref(sData), byref(sDataOut), c_bool(False))

0

In [630]:
#Dedicated command
 
# [OutParam   ] lx_wchar[10+1]
DLEDI_DEDICATED_GET_SDK_VERSION = "DLEDI_DEDICATED_GET_SDK_VERSION"

# [CommandData] lx_int32(0=Worker Data, 1=Intensity Table)
DLEDI_DEDICATED_FACTORY_RESET = "DLEDI_DEDICATED_FACTORY_RESET"

# [OutParam   ] bool(false=OFF, true=ON)
DLEDI_DEDICATED_GET_REMOCON_DISPLAY = "DLEDI_DEDICATED_GET_REMOCON_DISPLAY"
#[CommandData] bool(false=OFF, true=ON)
DLEDI_DEDICATED_SET_REMOCON_DISPLAY = "DLEDI_DEDICATED_SET_REMOCON_DISPLAY"

#[OutParam   ] lx_wchar[6+1]
DLEDI_DEDICATED_GET_CPU_VERSION = "DLEDI_DEDICATED_GET_CPU_VERSION"
#[OutParam   ] lx_wchar[4+1]
DLEDI_DEDICATED_GET_CPU_CHECKSUM = "DLEDI_DEDICATED_GET_CPU_CHECKSUM"
#[OutParam   ] lx_wchar[8+1]
DLEDI_DEDICATED_GET_CPU_PROGRAM = "DLEDI_DEDICATED_GET_CPU_PROGRAM"

#[OutParam   ] lx_int32(0=Sync, 1=Async, 2=Vis)
DLEDI_DEDICATED_GET_MODE = "DLEDI_DEDICATED_GET_MODE"
#[CommandData] lx_int32(0=Sync, 1=Async, 2=Vis)
DLEDI_DEDICATED_SET_MODE = "DLEDI_DEDICATED_SET_MODE"

# [OutParam   ] bool(false=Disable, true=Enable)
DLEDI_DEDICATED_GET_TRIGGER_ENABLE  = "DLEDI_DEDICATED_GET_TRIGGER_ENABLE"
#[CommandData] bool(false=Disable, true=Enable)
DLEDI_DEDICATED_SET_TRIGGER_ENABLE = "DLEDI_DEDICATED_SET_TRIGGER_ENABLE"

# [CommandData] lx_int32(Index:1->4) + lx_int32(Intensity:0->100)
# [OutParam   ] lx_int32[10](DA:0->1000)
DLEDI_DEDICATED_GET_DA_TABLE =  "DLEDI_DEDICATED_GET_DA_TABLE"
# [CommandData] lx_int32(Index:1->4) + lx_int32(Intensity:0->100) + lx_int32[10](DA:0->1000)
DLEDI_DEDICATED_SET_DA_TABLE = "DLEDI_DEDICATED_SET_DA_TABLE"

#[CommandData] lx_wchar[15+1]
DLEDI_DEDICATED_SET_REMOCON_TEXT = "DLEDI_DEDICATED_SET_REMOCON_TEXT"

# [CommandData] bool[4](false=Stay, true=Reverse)
DLEDI_DEDICATED_SET_SWITCH_RELATIVE = "DLEDI_DEDICATED_SET_SWITCH_RELATIVE"
# [CommandData] lx_int32[4](-100->100)
DLEDI_DEDICATED_SET_INTENSITY_RELATIVE = "DLEDI_DEDICATED_SET_INTENSITY_RELATIVE"

# [OutParam   ] lx_uint32(DelayTime:0->1000msec)
DLEDI_DEDICATED_GET_SWITCH_DELAY_TIME = "DLEDI_DEDICATED_GET_SWITCH_DELAY_TIME"
# [CommandData] lx_uint32(DelayTime:0->1000msec)
DLEDI_DEDICATED_SET_SWITCH_DELAY_TIME = "DLEDI_DEDICATED_SET_SWITCH_DELAY_TIME"

# [OutParam   ] bool(false=Hide, true=Show)
DLEDI_DEDICATED_GET_SHOW_SIMULATOR = "DLEDI_DEDICATED_GET_SHOW_SIMULATOR"
# [CommandData] bool(false=Hide, true=Show)
DLEDI_DEDICATED_SET_SHOW_SIMULATOR =  "DLEDI_DEDICATED_SET_SHOW_SIMULATOR"

# [CommandData] lx_uchar8(Command Data without Footer)
# [OutParam   ] lx_uchar8(Response Data without Footer)
DLEDI_DEDICATED_DIRECT_REQUEST  = "DLEDI_DEDICATED_DIRECT_REQUEST"

In [501]:
class CommandChar(Structure):
    _fields_ = [("Char", c_wchar)]
class DLEDI_Command(Structure):
    _fields_ = [("wszCommandString", CommandChar*256),
                ("pCommandData", c_void_p)]

In [626]:
# Enable external trigger
cmd = DLEDI_Command()
cmd_string = DLEDI_DEDICATED_SET_TRIGGER_ENABLE
for i in range(0, len(cmd_string)):
    cmd.wszCommandString[i].Char = cmd_string[i]
cmd.pCommandData = cast(pointer(c_bool(True)), c_void_p) # in order to avoid type dismatch
pUserOutParam = c_bool()
dledi.DLEDI_DedicatedCommand(ord(uiDeviceHandle.value), cmd, pUserOutParam)

0

In [627]:
# external trigger status
cmd = DLEDI_Command()
cmd_string = DLEDI_DEDICATED_GET_TRIGGER_ENABLE
for i in range(0, len(cmd_string)):
    cmd.wszCommandString[i].Char = cmd_string[i]
pUserOutParam = c_bool()
dledi.DLEDI_DedicatedCommand(ord(uiDeviceHandle.value), cmd, byref(pUserOutParam))
print(pUserOutParam)

c_bool(True)


In [631]:
# Get SDK Version
cmd = DLEDI_Command()
pUserOutParam = create_string_buffer(10+1)
cmd_string = DLEDI_DEDICATED_GET_SDK_VERSION
for i in range(0, len(cmd_string)):
    cmd.wszCommandString[i].Char = cmd_string[i]
dledi.DLEDI_DedicatedCommand(ord(uiDeviceHandle.value), cmd, byref(pUserOutParam))
print(pUserOutParam.raw)

b'2\x00.\x000\x00.\x000\x00.'


In [182]:
# disconnect to DLEDI, return 0 if success
ret = dledi.DLEDI_Close(ord(uiDeviceHandle.value))
if ret==0 :
    print("closed device")
else:
    print("fail to close device")

closed device
